# Folding

This notebooks shows a basic example of monomer folding with `prtm`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from functools import partial
from prtm import models
from prtm import visual
from prtm.query import caching

To get started let's define a simple protein sequence that we'd like to fold. It should be all uppercase; missing residues can be specified with `X`.

In [ ]:
sequence = "MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH"

## Monomer folding model comparison

Now we'll define a dict with all of the folding pipelines we want to try for this sequence. We'll avoid instantiating the pipelines for now to save memory. Upon instantiation model weights are downloaded to the default `torch.hub` directory (usually `~/.cache/torch/hub/checkpoints`) and the model is moved to the currently available `GPU`.

In [ ]:
folders = {
    # We can pass arguments with partial; we run AlphaFold models
    # using the Uni-Fold codebase which is written in pytorch
    # This is the only model that currently supports templates
    "AlphaFold": partial(models.UniFoldForFolding, model_name="model_1_af2", use_templates=True),
    "OpenFold": models.OpenFoldForFolding,
    "OmegaFold": models.OmegaFoldForFolding,
    "RoseTTAFold": models.RoseTTAFoldForFolding,
    "ESMFold": models.ESMForFolding,
    "DMPFold": models.DMPFoldForFolding,
}

Four of the models we're using require MSAs for inference (`AlphaFold`, `OpenFold`, `RoseTTAFold`, `DMPFold`). `prtm` will perform MSA queries automatically using either `MMSeqs2` or `jackhmmer` and the databases released with `AlphaFold`. Currently running inference with the AlphaFold weights only supports `MMSeqs` for MSAs. This is preferable because `MMSeqs` is significantly faster; eventually all models that use MSAs will default to it. Once queries are completed the results are cached locally in a simple `sqlite` database. Any folding pipelines that require MSAs will first check the cache before recomputing. By default the caching is stored in `~/.prtm/queries.db`. Caching is responsive to both the input sequence and the parameters used in the querying pipeline. To save time for this example, we'll use pre-computed MSAs by changing the default cache path.

In [ ]:
caching.set_db_path("./cached_queries_for_testing.db")

For simplicity we're mostly using default model weights for each folding pipeline; however, some have multiple options that can be tested. We can get a list to choose from with the `available_models` property.

In [ ]:
# List models to choose from for models.UniFoldForFolding
print("UniFold model names:", models.UniFoldForFolding.available_models)

Let's do some folding! Every pipeline in `prtm` has at least two outputs. The last output is always a dictionary of `aux_outputs` that include things like confidence scores, loss metrics, etc. The first output of folding models is a protein structure class that we'll discuss in detail in another notebook.

In [ ]:
predicted_structures = {}
aux_outputs = {}
for folder_name, fold_pipeline in folders.items():
    print(f"Running {folder_name}...")
    # Initialize the folder model with defaults
    folder = fold_pipeline()
    # Run pipelines with the __call__ method
    pred_structure, aux = folder(sequence)
    predicted_structures[folder_name] = pred_structure
    aux_outputs[folder_name] = aux

Let's check the outputs.

In [ ]:
aux_outputs

In [ ]:
predicted_structures

As described the `aux_outputs` just contain global measures of structure confidence. The predicted strctures are a few different kinds of `protein` classes. The number after `Protein` defines the number of atoms in the structure. In this case, `Protein37` and `Protein14` are two different ways of representing a protein structure with sidechains included while `Protein5` and `Protein4` are structures that only include atoms in the backbone (`N`, `CA`, `C`, `O`, `CB`). To view any structure with per-residue confidence predictions we simply call `show` on the structure.

In [ ]:
# A structure with sidechains
# We can color the structure with any matplotlib colormap
predicted_structures["OmegaFold"].show(cmap="jet")

In [ ]:
# A structure without sidechains
predicted_structures["RoseTTAFold"].show(cmap="jet")

Really for this kind of comparison we'd like to see the structures together in a single figure. There are two options. First, we can superimpose any two structures:

In [ ]:
# The first structure is shown with some opacity for ease of visualization
visual.view_superimposed_structures(
    predicted_structures["AlphaFold"], predicted_structures["ESMFold"], color1="green"
)

Second we can view all of the structures in a grid with locked views:

In [ ]:
# When working with a mixture of structures that don't all have
# sidechains it's usually better to turn them off
visual.view_aligned_structures_grid(
    list(predicted_structures.values()), cmap="jet", show_sidechains=False, max_grid_cols=3
)

Finally we can export any `protein` structure to `PDB`. Behind-the-scenes `prtm` will ensure that structures with and without sidechains are written correctly so there's no need to do any manual conversions.

In [ ]:
for folder_name, pred_structure in predicted_structures.items():
    with open(f"{folder_name}_prediction.pdb", mode="w") as f:
        f.writelines(pred_structure.to_pdb())

## Multimer Folding

Currently, we support two multimer folding models: `AlphaFold-Multimer` and `UniFold-Multimer`. Both are implemented in `Uni-Fold` within `prtm` and they usually produce very similar results because `Uni-Fold` is a clone of the `AlphaFold` codebase into `PyTorch`.

For multiple sequences, we can pass a list of sequences or a dict indexed by chain id.

In [ ]:
complex_sequence_a = (
    "TTPLVHVASVEKGRSYEDFQKVYNAIALKLREDDEYDNYIGYGPVLVRLAWHTSGTW"
    "DKHDNTGGSYGGTYRFKKEFNDPSNAGLQNGFKFLEPIHKEFPWISSGDLFSLGGVTA"
    "VQEMQGPKIPWRCGRVDTPEDTTPDNGRLPDADKDADYVRTFFQRLNMNDREVVALMGAH"
    "ALGKTHLKNSGYEGPWGAANNVFTNEFYLNLLNEDWKLEKNDANNEQWDSKSGYMMLPTDY"
    "SLIQDPKYLSIVKEYANDQDKFFKDFSKAFEKLLENGITFPKDAPSPFIFKTLEEQGL"
)
complex_sequence_b = (
    "TEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHSGQAEGYSYTDA"
    "NIKKNVLWDENNMSEYLTNPKKYIPGTKMAIGGLKKEKDRNDLITYLKKACE"
)
complex_sequence_dict = {"A": complex_sequence_a, "B": complex_sequence_b}

In [ ]:
multimer_folders = {
    "AlphaFold": partial(models.UniFoldForFolding, model_name="multimer_1_af2_v3", use_templates=True),
    "Uni-Fold": partial(models.UniFoldForFolding, model_name="multimer_ft", use_templates=True),
}

In [ ]:
predicted_complexes = {}
aux_outputs = {}
for folder_name, fold_pipeline in multimer_folders.items():
    print(f"Running {folder_name}...")
    folder = fold_pipeline()
    pred_complex, aux = folder(complex_sequence_dict)
    predicted_complexes[folder_name] = pred_complex
    aux_outputs[folder_name] = aux

In [ ]:
aux_outputs

Let's see how the predicted complexes compare to each other.

In [ ]:
visual.view_aligned_structures_grid(
    list(predicted_complexes.values()), cmap="jet", show_sidechains=True
)

## Symmetric Folding

Instead of folding a symmetric protein complex as a set of independent chains, we can instead fold them as a single chain with multiple copies. In short, we give the fold a single sequence and get out a symmetric complex. Only `Uni-Fold Symmetry` supports this.

In [ ]:
symmetric_sequence = (
    "PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKASCLYGQLPKFQDGD"
    "LTLYQSNTILRHLGRTLGLYGKDQQEAALVDMVNDGVEDLRCKYISLIYTNYEAGKDDYV"
    "KALPGQLKPFETLLSQNQGGKTFIVGDQISFADYNLLDLLLIHEVLAPGCLDAFPLLSAY"
    "VGRLSARPKLKAFLASPEYVNLPINGNGKQ"
)

We'll fold the complex with `C2` symmetry to get two identical chains situated in a complex.

In [ ]:
sym_folder = models.UniFoldForFolding(
    model_name="uf_symmetry", use_templates=True, random_seed=0, symmetry_group="C2"
)

In [ ]:
sym_structure, sym_aux_output = sym_folder(
    symmetric_sequence, max_recycling_iters=3, num_ensembles=2
)

In [ ]:
# Confidence score for the folded structure
sym_aux_output

In [ ]:
sym_structure.show()

## Conformation Sampling

All the folding models we've looked at so far are (nearly) deterministic. We can sample possible conformations by using `EigenFold` instead. `EigenFold` is built on top of `OmegaFold` but adds a sampling procedure during structure decoding.

In [ ]:
fold_sampler = models.EigenFoldForFoldSampling(random_seed=0)

In [ ]:
sampled_structures = []
sampled_aux_outputs = []
for _ in range(5):
    sampled_structure, sampled_aux = fold_sampler(sequence)
    sampled_structures.append(sampled_structure)
    sampled_aux_outputs.append(sampled_aux)

In [ ]:
sampled_aux_outputs

In [ ]:
sampled_structures

This time we get `elbo` values which in this case is a measure of the likelihood for a structure. The structure returned by `EigenFold` are `CA` traces which means that only a single backbone atom per residue is predicted. The visualization tools for these structures are a bit different, but we can still call the `show` method to view them.

In [ ]:
# We can't specify pyplot colormaps any more, we just get basic color names of can pass HEX colors
sampled_structures[0].show(cmap="green")

Like before we can superimpose structures to see more easily where they differ.

In [ ]:
visual.view_superimposed_ca_traces(sampled_structures)

Although this is a very simple structure, this comparison shows us where parts of the structure that are likely less stable (the results overlap nicely with the confidence predictions of the other folding models).

## Antibody Folding

For the particular use case of holding antibodies, which can have a heavy and light chain, there's `IgFold`. `IgFold` has been trained on a large dataset of antibody structures from a few different species.

Unlike the other folding algorithms, we provide a dictionary of sequences to `IgFold`. We can only supply 2 sequences: one each for a heavy and light chain. It's also acceptable to pass a single sequence (`H` or `L`).

In [ ]:
sequences = {
    "H": "EVQLVQSGPEVKKPGTSVKVSCKASGFTFMSSAVQWVRQARGQRLEWIGWIVIGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAPYCSSISCNDGFDIWGQGTMVTVS",
    "L": "DVVMTQTPFSLPVSLGDQASISCRSSQSLVHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQSTHVPYTFGGGTKLEIK",
}

In [ ]:
antibody_folder = models.IgFoldForFolding()

In [ ]:
antibody_structure, antibody_aux_output = antibody_folder(sequences)

In [ ]:
# The confidence predictions from IgFold are actually RMSD predictions so they aren't scaled
# from 0-100 like a probability
antibody_structure.show(bfactor_is_confidence=False)

In [ ]:
# IgFold applies a gradient-based structure refinement, the score is a measure
# of the energy of the structure: lower is better.
antibody_aux_output